# Finite Time Lyapunov Exponent (FTLE) analysis
### Author: Justine John (JJ) A. Serdoncillo 
### Date Created: August 7, 2023
---

# <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
1. [Yuanhang Functions](#1)
2. [Read Data](#2)
3. [Particle Tracking](#3)
4. [FTLE Calculation](#4)
5. [Visualization](#5)
6. [Save Data](#6)

## PIV Experimental data for hydrofoil pitching at the leading edge at $Re = 10,000$ and $St = 0.25$

### Airfoil kinematics: $\theta = \theta_0 + \Delta \theta sin(2\pi f^*t)$, where 
***
***

## <font color='289C4E'>Yuanhang Functions<font><a id='1'></a>


In [1]:
# Read the PIV velocity data of a 2D pitching foil
# Calculate the vorticity and Q value
# Plot the vorticity, Q, influence potential, and vorticity-induced force/moment density
# Yuanhang Zhu
# Aug 1, 2023

using DelimitedFiles
using Plots

function calculate_vorticity_Q(x::Matrix{T}, y::Matrix{T}, u::Matrix{T}, v::Matrix{T}) where T
    nx, ny = size(u,2), size(u,1)
    vorticity = similar(u, T)
    q_value = similar(u, T)
    
    for j in 2:(ny-1)
        for i in 2:(nx-1)
            dv_dx = (v[j, i+1] - v[j, i-1]) / (x[j, i+1] - x[j, i-1])
            du_dy = (u[j+1, i] - u[j-1, i]) / (y[j+1, i] - y[j-1, i])
            
            # Calculate rate of strain tensor components
            ds_dx = 0.5 * (du_dy + dv_dx)
            du_dx = (u[j, i+1] - u[j, i-1]) / (x[j, i+1] - x[j, i-1])
            dv_dy = (v[j+1, i] - v[j-1, i]) / (y[j+1, i] - y[j-1, i])
            ds_dy = 0.5 * (du_dx + dv_dy)
            
            # Calculate vorticity
            vorticity[j, i] = dv_dx - du_dy

            # Calculate Q values
            # q_value[j, i] = 0.5 * (vorticity[j, i]^2 + ds_dx^2 + ds_dy^2)
            q_value[j, i] = -du_dy * dv_dx + du_dx * dv_dy
        end
    end
    
    return vorticity, q_value
end

num = 1; # phase number

# num phase number goes from 1 to 50
## 4 columns and 9639 rows for num 1 = 119*81


### Load Airfoil Coordinates - changed directory

In [2]:
#cd("/Users/yuanhangzhu/Dropbox/Research/UVA/FMPM/2D PIV case/foil shape") # load foil coordinates
cd("E:/Research/2D_PIV_pitching_foil_YZ/foil shape") # load foil coordinates

shape = readdlm("foil_coordinate_$num.txt", ',', '\n')
xfoil = shape[:,1];
yfoil = shape[:,2];

### Load Velocity Field Data - changed directory

In [3]:
#cd("/Users/yuanhangzhu/Dropbox/Research/UVA/FMPM/2D PIV case/PIV data") # load velocity field data
cd("E:/Research/2D_PIV_pitching_foil_YZ/PIV data") # load velocity field data

velocity_data = readdlm("velocity_field_$num.txt", ',', '\n')

xn = 119; # number of values in x-direction
yn = 81; # number of values in y-direction

x = velocity_data[:,1];
y = velocity_data[:,2];
u = velocity_data[:,3];
v = velocity_data[:,4];

xdata = reshape(x, (xn, yn));
ydata = reshape(y, (xn, yn));
udata = reshape(u, (xn, yn));
vdata = reshape(v, (xn, yn));


vorticity, q_value = calculate_vorticity_Q(ydata, xdata, -vdata, -udata);

In [49]:
x_range = xdata[:,1]
y_range = transpose(ydata[1,:])
print("DONE")

DONE

## <font color='289C4E'>Read Data<font><a id='2'></a>
### Reading all the data at once 

In [11]:
cd("E:/Research/2D_PIV_pitching_foil_YZ/PIV data")
velocity_data = readdlm("velocity_field_1.txt", ',', '\n')
x = velocity_data[:,1];
y = velocity_data[:,2];

xn = 119; # number of values in x-direction
yn = 81; # number of values in y-direction

xdata = reshape(x, (xn, yn));
ydata = reshape(y, (xn, yn));

tstep_start = 1
tstep_end = 50
og_num_tsteps = 50
og_tstep_delta = 1
inter = 10
num_tsteps = (og_num_tsteps-1)*inter+1
tstep_delta = og_tstep_delta/inter
Vel_contour = zeros(Float64, 2, num_tsteps, xn, yn)

for i in 1:og_num_tsteps-1
    velocity_data = readdlm("velocity_field_$num.txt", ',', '\n')
    u1 = velocity_data[:,3];
    v1 = velocity_data[:,4];
    velocity_data = readdlm("velocity_field_$(num+1).txt", ',', '\n')
    u2 = velocity_data[:,3];
    v2 = velocity_data[:,4];
    
    xdata = reshape(x, (xn, yn));
    ydata = reshape(y, (xn, yn));
    udata = reshape(u, (xn, yn));
    vdata = reshape(v, (xn, yn));
    
    for p in 0:inter-1
        uu = u1*(inter-p)/inter + u2*(p)/inter
        vv = v1*(inter-p)/inter + v2*(p)/inter
        Vel_contour[1, inter*(i-1)+p+1, :, :] =  reshape(uu,(xn, yn))
        Vel_contour[2, inter*(i-1)+p+1, :, :] =  reshape(vv,(xn, yn))
    end
    
    if i == og_num_tsteps-1
        Vel_contour[1, num_tsteps, :, :] =  reshape(u2,(xn, yn))
        Vel_contour[2, num_tsteps, :, :] =  reshape(v2,(xn, yn))
    end
    
end

### Plot Vorticity Field

<div class="alert alert-block alert-danger">  
could not load library "libGR.dll" <br>
The specified module could not be found. 
</div>
tried to resolve by  <br>

import Pkg <br>
ENV["GRDIR"]="" <br>
Pkg.build("GR")<br>

but did not end up working still, and received error    

<div class="alert alert-block alert-warning">  
Error building `GR`: <br>
[ Info: Downloading pre-compiled GR latest Windows binary <br>
ERROR: LoadError: IOError: could not spawn `'C:\Users\GreenFluids_VR\Desktop\julia-1.9.2\bin\..\libexec/7z' x downloads/gr- <br>latest-Windows-x86_64.tar.gz -y`: no such file or directory (ENOENT)
</div>

<div class="alert alert-block alert-success">  
Following issue above is resolved when run with hp
</div>
<div class="alert alert-block alert-danger">  
Issue still persists with vr 
</div>

In [4]:
import Pkg
ENV["GRDIR"]=""
Pkg.build("GR")
println("DONE")

Precompiling project...
  ✓ LinearSolve → LinearSolveIterativeSolversExt
  ✓ NonlinearSolve
  ✓ OrdinaryDiffEq
  ✓ ConstrainedSystems
  ✓ ImmersedLayers


DONE


  5 dependencies successfully precompiled in 299 seconds. 364 already precompiled. 1 skipped during auto due to previous errors.


In [ ]:
plot(range(1,5),range(1,5))

In [ ]:
#p1 = 
#contourf(xdata[2:end-1,1], ydata[1,2:end-1], max.(min.(vorticity[2:end-1,2:end-1]', 30), -30), levels=100, lw=0, xlabel="x (m)", ylabel="y (m)", title="Vorticity", color=:vik, aspect_ratio=:equal, clim=(-30,30), xlims=(-0.1, 0.2), ylims=(-0.1,0.1));
#plot(xfoil,yfoil,fill=true,aspect_ratio=:equal,c=:black,legend = false)

In [6]:
p2 = contourf(xdata[2:end-1,1], ydata[1,2:end-1], max.(min.(q_value[2:end-1,2:end-1]', 100), -100), levels=100, lw=0, xlabel="x (m)", ylabel="y (m)", title="Q", color=:vik, aspect_ratio=:equal, clim=(-100,100), xlims=(-0.1, 0.2), ylims=(-0.1,0.1));
plot!(xfoil,yfoil,fill=true,aspect_ratio=:equal,c=:black,legend = false)

GKS: svgplugin.dll: can't load library, error 126 (0x7e)
GKS: cairoplugin.dll: can't load library, error 126 (0x7e)


### Load Phi Field data - changed directory

In [ ]:
#cd("/Users/yuanhangzhu/Dropbox/Research/UVA/FMPM/2D PIV case/Potential/drag_potential")
cd("E:/Research/2D_PIV_pitching_foil_YZ/Potential/drag_potential")
phi_drag = readdlm("phi_drag_$num.txt", ',', '\n')[:,3]; # load phi field data
phi_drag_data = reshape(phi_drag, (yn, xn))

p3 = contourf(xdata[2:end-1,1], ydata[1,2:end-1], phi_drag_data[2:end-1,2:end-1], levels=50, lw=0, xlabel="x (m)", ylabel="y (m)", title="Phi", color=:vik, aspect_ratio=:equal, clim=(-0.01,0.01), xlims=(-0.1, 0.2), ylims=(-0.1,0.1));
plot!(xfoil,yfoil,fill=true,aspect_ratio=:equal,c=:black,legend = false)

In [ ]:
drag_density = -2*.*(q_value',phi_drag_data) # calculate vorticity-induced force density
p3 = contourf(xdata[2:end-1,1], ydata[1,2:end-1], max.(min.(drag_density[2:end-1,2:end-1], 0.2), -0.2), levels=50, lw=0, xlabel="x (m)", ylabel="y (m)", title="Drag density", color=:vik, aspect_ratio=:equal, clim=(-0.2,0.2), xlims=(-0.1, 0.2), ylims=(-0.1,0.1));
plot!(xfoil,yfoil,fill=true,aspect_ratio=:equal,c=:black,legend = false)

## <font color='289C4E'>Particle Tracking and FTLE calculation<font><a id='3'></a>

In [12]:
function find_me(xval, yval)
    x1 = argmin(abs.(x_range .- xval))
    y1 = argmin(abs.(y_range .- yval))

    if x1 >= size(x_range)[1]-1
        x1 -= 3
    end
    
    if y1 >= size(y_range)[1]-1
        y1 -= 3
    end

    x2 = x1 + 1
    y2 = y1 + 1
    
    return x1, x2, y1, y2
end

find_me (generic function with 1 method)

In [16]:
tstep_start = 1
tstep_end = 50

50

In [27]:
tstep_delta = 0.1

0.1

In [50]:
function FTLE(nx,ny,x0,x1,y0,y1,t_0,t_1)
    move = sign(t_1-t_0)
    tstep_range = collect(range(tstep_start, tstep_end, length=num_tsteps))
    t_0_ind = argmin(abs.(tstep_range .- t_0))
    t_1_ind = argmin(abs.(tstep_range .- t_1))
    tsteps = Int64(abs(t_1 - t_0)/tstep_delta + 1)
    x = collect(range(x0, x1, length=nx))
    y = collect(range(y0, y1, length=ny))
    particles = zeros(3, tsteps+1, nx, ny)

    for ii in 1:nx, jj in 1:ny
        particles[1,1,ii, :] .= x[ii]
        particles[2,1, :,jj] .= y[jj]
    end
    
    #=
    for ii in 1:nx, jj in 1:ny
        xind = argmin(abs.(y_range.-particles[2,1,ii,jj]))
        yind = argmin(abs.(x_range.-particles[1,1,ii,jj]))
        if Bvo_contour[1,1,xind,yind] == 1
            particles[:,1,ii,jj] .= NaN
        end
    end
    =#
    
    x = copy(x_range)
    y = copy(y_range)
    # Iterate through all timesteps 
    count = 1
    for tt in t_0_ind:move:t_1_ind
        for ii in 1:nx, jj in 1:ny
            #if any(isnan,particles[1:2,count,ii,jj]) == false 
            # Let's use Runge Kutta 4th order
            kxy = zeros(4,2)
            a = particles[1,move*(tt-t_0_ind)+1,ii,jj]
            b = particles[2,move*(tt-t_0_ind)+1,ii,jj]
            #=Runge Kutta 4th Order K1 =#
            c = copy(a)
            d = copy(b)
            x1, x2, y1, y2 = find_me(c, d)
            for UV in 1:2
                kxy[1,UV] = 1/((y[y2]-y[y1])*(x[x2]-x[x1])) * ( 
                               (x[x2]-c)*(y[y2]-d)*Vel_contour[UV,tt,y1,x1] + 
                               (c-x[x1])*(y[y2]-d)*Vel_contour[UV,tt,y2,x1] +
                               (x[x2]-c)*(d-y[y1])*Vel_contour[UV,tt,y1,x2] +
                               (c-x[x1])*(d-y[y1])*Vel_contour[UV,tt,y2,x2] )
            end
            #=Runge Kutta 4th Order K2 =#
            c = a + 0.5*kxy[1,1]*tstep_delta/1000
            d = b + 0.5*kxy[1,2]*tstep_delta/1000
            x1, x2, y1, y2 = find_me(c, d)
            for UV in 1:2
                kxy[2,UV] = 1/((y[y2]-y[y1])*(x[x2]-x[x1])) * ( 
                               (x[x2]-c)*(y[y2]-d)*0.5*(Vel_contour[UV,tt,y1,x1]+Vel_contour[UV,tt+move,y1,x1]) + 
                               (c-x[x1])*(y[y2]-d)*0.5*(Vel_contour[UV,tt,y2,x1]+Vel_contour[UV,tt+move,y2,x1]) +
                               (x[x2]-c)*(d-y[y1])*0.5*(Vel_contour[UV,tt,y1,x2]+Vel_contour[UV,tt+move,y1,x2]) +
                               (c-x[x1])*(d-y[y1])*0.5*(Vel_contour[UV,tt,y2,x2]+Vel_contour[UV,tt+move,y2,x2]) )
            end
            #=Runge Kutta 4th Order K3 =#
            c = a + 0.5*kxy[2,1]*tstep_delta/1000
            d = b + 0.5*kxy[2,2]*tstep_delta/1000
            x1, x2, y1, y2 = find_me(c, d)
            for UV in 1:2
                kxy[3,UV] = 1/((y[y2]-y[y1])*(x[x2]-x[x1])) * ( 
                               (x[x2]-c)*(y[y2]-d)*0.5*(Vel_contour[UV,tt,y1,x1]+Vel_contour[UV,tt+move,y1,x1]) + 
                               (c-x[x1])*(y[y2]-d)*0.5*(Vel_contour[UV,tt,y2,x1]+Vel_contour[UV,tt+move,y2,x1]) +
                               (x[x2]-c)*(d-y[y1])*0.5*(Vel_contour[UV,tt,y1,x2]+Vel_contour[UV,tt+move,y1,x2]) +
                               (c-x[x1])*(d-y[y1])*0.5*(Vel_contour[UV,tt,y2,x2]+Vel_contour[UV,tt+move,y2,x2]) )
            end
            #=Runge Kutta 4th Order K4 =#
            c = a + kxy[3,1]*tstep_delta/1000
            d = b + kxy[3,2]*tstep_delta/1000
            x1, x2, y1, y2 = find_me(c, d)
            for UV in 1:2
                kxy[4,UV] = 1/((y[y2]-y[y1])*(x[x2]-x[x1])) * ( 
                               (x[x2]-c)*(y[y2]-d)*Vel_contour[UV,tt+move,y1,x1] + 
                               (c-x[x1])*(y[y2]-d)*Vel_contour[UV,tt+move,y2,x1] +
                               (x[x2]-c)*(d-y[y1])*Vel_contour[UV,tt+move,y1,x2] +
                               (c-x[x1])*(d-y[y1])*Vel_contour[UV,tt+move,y2,x2] )
            end
            for UV in 1:2
                particles[UV,move*(tt-t_0_ind)+2,ii,jj] = particles[UV,move*(tt-t_0_ind)+1,ii,jj] + sign(move)*(kxy[1,UV]+2*kxy[2,UV]+2*kxy[3,UV]+kxy[4,UV])*tstep_delta/6000
            end
            #end
        end
        count += 1
    end
    
    for tt in 1:tsteps
        for ii in 2:nx-1
            for jj in 2:ny-1
                ul = (particles[1,tt,ii+1,jj]-particles[1,tt,ii-1,jj])/(particles[1,1,ii+1,jj]-particles[1,1,ii-1,jj])
                ur = (particles[2,tt,ii+1,jj]-particles[2,tt,ii-1,jj])/(particles[2,1,ii,jj+1]-particles[2,1,ii,jj-1])
                ll = (particles[1,tt,ii,jj+1]-particles[1,tt,ii,jj-1])/(particles[1,1,ii+1,jj]-particles[1,1,ii-1,jj])
                lr = (particles[2,tt,ii,jj+1]-particles[2,tt,ii,jj-1])/(particles[2,1,ii,jj+1]-particles[2,1,ii,jj-1])
                B = [ul ur; ll lr]
                A = transpose(B) * B
                G = eigvals(A)
                H = ones(length(G))
                for i in 1:length(H)
                    H[i] = abs(G[i])
                end
                sigma = maximum(H)
                particles[3,tt+1,ii,jj] = 0.5/tstep_delta*log(sigma)
            end
        end
    end

    
    flow = particles[:,:,2:nx-1,2:ny-1]
    println("done")

    return flow
end

FTLE (generic function with 1 method)

In [51]:
p300 = FTLE(10,10,-0.1,0.2,-0.1,0.1, 1,25)
n300 = FTLE(10,10,-0.1,0.2,-0.1,0.1,49,25)
print("DONE")

LoadError: MethodError: no method matching isless(::Int64, ::CartesianIndex{2})

[0mClosest candidates are:
[0m  isless(::Real, [91m::AbstractGray[39m)
[0m[90m   @[39m [36mColorTypes[39m [90mC:\Users\jjser\.julia\packages\ColorTypes\1dGw6\src\[39m[90m[4moperations.jl:39[24m[39m
[0m  isless([91m::CartesianIndex{N}[39m, ::CartesianIndex{N}) where N
[0m[90m   @[39m [90mBase[39m [90m[4mmultidimensional.jl:124[24m[39m
[0m  isless(::Real, [91m::AbstractFloat[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:178[24m[39m
[0m  ...
